In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
import warnings
warnings.filterwarnings("ignore")

from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import pymysql

import json
from tools import *

qm = QHJMall(MALL_KEY)


import datetime

sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID', '发货商','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})



In [18]:
l = []
for order in qm.gen_orders("original"):
    l.append(order)


请求 {'order_status': 0, 'order_time': '', 'order_id': '', 'order_type': 'original', 'page': 1}


In [30]:


data = pd.DataFrame(l)
data = pd.concat([data['用户信息'].str.extract("""
        (?<=下单人：)(?P<收件人>[^\n]+)
        \s+手机：(?P<联系方式>[^\n]+)
        \s+收货地址：(?P<收货地址>[^\n]+)""", flags=re.S | re.X),data],axis=1)
data = data.replace({"支付金额": {"￥": ""}}, regex=True).fillna({"支付金额": 0})

data['支付金额'] = data['支付金额'].astype(int)

# 拆分为原子
data = data.drop(columns='订单详情').join(data['订单详情'].str.split('\n',expand=True).stack().reset_index(level=1, drop=True).rename("详情"))


data = pd.concat([data[['订单号', '收件人','联系方式','收货地址', '用户信息','下单时间','备注','支付金额','支付时间']], data['详情'].str.extract(
"""
(?P<商品名>.+?)(?=（规格[：:])（
规格[：:](?P<规格>.+?)(?=[，,]商品ID[：:])
[，,]商品ID[：:](?P<商品ID>.+)）\sX(?P<数量>\s\d+)""",flags=re.X)],axis=1)

data = pd.merge(data,sku_goods[['商品ID','goods_type','单价','发货商']],on="商品ID")
data['数量'] = data['数量'].astype(int)
data['支付金额'] = data['数量'] * data['单价']

In [ ]:
["订单号", '商品ID', "发货商", "数量", "支付金额", "备注", "收件人", "联系方式", "收货地址", "goods_type",
                  "导出订单时间", '下单时间', '支付时间']

In [31]:
set(["订单号", '商品ID', "发货商", "数量", "支付金额", "备注", "收件人", "联系方式", "收货地址", "goods_type",
                  "导出订单时间", '下单时间', '支付时间']) - set(data.columns)

{'导出订单时间'}

In [20]:
data['支付金额'].sum()

207245